# Install pgvector to Postgre SQL 
- Follow the git hub https://github.com/pgvector/pgvector to build and install the vector extension for PostgreSQL server
- Create the vector extension in PostgreSQL via admin user
- Get started with example in python https://github.com/pgvector/pgvector-python/blob/master/examples/openai/example.py

- In Python, also need to install `pgvector` package

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()

for key, value in os.environ.items():
    print(f"{key} = {value}")


ALLUSERSPROFILE = C:\ProgramData
APPDATA = C:\Users\Trung Thao\AppData\Roaming
APPLICATIONINSIGHTS_CONFIGURATION_CONTENT = {}
APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL = 1
APPLICATION_INSIGHTS_NO_STATSBEAT = true
BROWSERLESS_TOKEN = 2Sh7jVShj9uozba474ea5989877be1a61a17b7a22b58e0e9f
CHOCOLATEYINSTALL = C:\ProgramData\chocolatey
CHOCOLATEYLASTPATHUPDATE = 133333699339095704
CHROME_CRASHPAD_PIPE_NAME = \\.\pipe\crashpad_9240_MGLMXVBRLRHNZBLP
COMMONPROGRAMFILES = C:\Program Files\Common Files
COMMONPROGRAMFILES(X86) = C:\Program Files (x86)\Common Files
COMMONPROGRAMW6432 = C:\Program Files\Common Files
COMPUTERNAME = DTT-PC
COMSPEC = C:\WINDOWS\system32\cmd.exe
DRIVERDATA = C:\Windows\System32\Drivers\DriverData
EFC_20896_1262719628 = 1
EFC_20896_1592913036 = 1
EFC_20896_2283032206 = 1
EFC_20896_2775293581 = 1
EFC_20896_3789132940 = 1
ELECTRON_RUN_AS_NODE = 1
FPS_BROWSER_APP_PROFILE_STRING = Internet Explorer
FPS_BROWSER_USER_PROFILE_STRING = Default
GOOGLE_API_KEY = AIzaSyDDvdxqxVkwqPAL

In [5]:
# %pip install pgvector

import os
import numpy as np

import psycopg2
from pgvector.psycopg2 import register_vector

# Connect to PostgreSQL (update credentials as needed)
conn = psycopg2.connect(
    dbname="netflix_db",
    user=os.getenv("POSTGRES_USERNAME_DB", "postgres"),
    password=os.getenv("POSTGRES_PASSWORD_DB", "Post!234"),
    host=os.getenv("POSTGRES_DATABASE_HOST", "localhost"),
    port=os.getenv("POSTGRES_DATABASE_PORT", "5432")
)

cur = conn.cursor()
cur.execute('CREATE EXTENSION IF NOT EXISTS vector')
register_vector(conn)

cur.execute('DROP TABLE IF EXISTS documents')
cur.execute('CREATE TABLE documents (id bigserial PRIMARY KEY, content text, embedding vector(1536))')
conn.commit()
cur.close()




Insert rows into `documents` table

In [ ]:
from openai import OpenAI
client = OpenAI()

def embed(input):
    response = client.embeddings.create(input=input, model='text-embedding-3-small')
    return [v.embedding for v in response.data]


input = [
    'The dog is barking',
    'The cat is purring',
    'The bear is growling'
]
embeddings = embed(input)
conn.rollback()  # Reset transaction state if previous error occurred
cur = conn.cursor()
for content, embedding in zip(input, embeddings):
    cur.execute('INSERT INTO documents (content, embedding) VALUES (%s, %s)', (content, np.array(embedding)))
conn.commit()
cur.close()


Make search to find text similar with query = 'tiger'

In [8]:
query = 'tiger'
query_embedding = embed([query])[0]
cur = conn.cursor()

cur.execute('SELECT content FROM documents ORDER BY embedding <=> %s LIMIT 5', (np.array(query_embedding),))
result = cur.fetchall()
for row in result:
    print(row[0])
cur.close()

The bear is growling
The cat is purring
The dog is barking
